In [7]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from searoute import searoute
import folium

# Step 1: Load the port data from CSV
# Using the actual column names from your provided data
def load_port_data(filename):
    """Load port data from CSV file."""
    try:
        ports_df = pd.read_csv(filename)
        
        # Keep only necessary columns and rename for easier use
        ports_df = ports_df[['Main Port Name', 'Latitude', 'Longitude', 'Country Code']]
        ports_df.rename(columns={
            'Main Port Name': 'port_name',
            'Latitude': 'latitude',
            'Longitude': 'longitude',
            'Country Code': 'country'
        }, inplace=True)
        
        # Filter out ports with missing coordinates
        ports_df = ports_df.dropna(subset=['latitude', 'longitude'])
        
        print(f"Loaded {len(ports_df)} ports with valid coordinates")
        return ports_df
    
    except Exception as e:
        print(f"Error loading port data: {e}")
        return None

# Step 2: Select random ports for route generation
def generate_random_routes(ports_dataframe, num_vessels=1):
    """Generate random routes by selecting start and end ports."""
    routes = []
    
    for vessel_id in range(1, num_vessels + 1):
        # Randomly select two different ports
        selected_ports = ports_dataframe.sample(n=2).reset_index(drop=True)
        
        start_port = {
            'name': selected_ports.loc[0, 'port_name'],
            'country': selected_ports.loc[0, 'country'] if 'country' in selected_ports.columns else '',
            'lat': selected_ports.loc[0, 'latitude'],
            'lon': selected_ports.loc[0, 'longitude']
        }
        
        end_port = {
            'name': selected_ports.loc[1, 'port_name'],
            'country': selected_ports.loc[1, 'country'] if 'country' in selected_ports.columns else '',
            'lat': selected_ports.loc[1, 'latitude'],
            'lon': selected_ports.loc[1, 'longitude']
        }
        
        routes.append({
            'vessel_id': f"Vessel_{vessel_id}",
            'start_port': start_port,
            'end_port': end_port
        })
        
        print(f"Generated route for Vessel_{vessel_id}: {start_port['name']} to {end_port['name']}")
    
    return routes

# Step 3: Generate realistic routes using searoute-py
def generate_searoutes(routes):
    """Generate realistic sea routes using searoute-py."""
    for route in routes:
        start = (float(route['start_port']['lon']), float(route['start_port']['lat']))
        end = (float(route['end_port']['lon']), float(route['end_port']['lat']))
        
        try:
            # Generate route using searoute
            print(f"Generating route from {start} to {end}...")
            route_geometry = searoute(start, end)
            
            # Extract coordinates from the route geometry
            if route_geometry and 'geometry' in route_geometry:
                route['route_coordinates'] = route_geometry['geometry']['coordinates']
                print(f"Route generated for {route['vessel_id']} from {route['start_port']['name']} to {route['end_port']['name']} with {len(route['route_coordinates'])} points")
            else:
                print(f"Failed to generate route for {route['vessel_id']} - Invalid geometry returned")
                route['route_coordinates'] = []
        except Exception as e:
            print(f"Error generating route: {e}")
            route['route_coordinates'] = []
    
    return routes

# Step 4: Visualize the routes using Folium
def visualize_routes(routes, output_file="vessel_routes.html"):
    """Create an interactive map visualization of the routes."""
    # Create a map centered at a middle point
    map_center = [0, 0]  # Default center
    
    if routes and len(routes) > 0 and 'route_coordinates' in routes[0]:
        # Find a better center if we have routes
        lats = []
        lons = []
        for route in routes:
            if route['route_coordinates']:
                for coord in route['route_coordinates']:
                    lons.append(coord[0])
                    lats.append(coord[1])
        
        if lats and lons:
            map_center = [sum(lats)/len(lats), sum(lons)/len(lons)]
    
    # Create the map
    m = folium.Map(location=map_center, zoom_start=2)
    
    # Add routes to the map
    for route in routes:
        # Add start port marker
        start_popup = f"Start: {route['start_port']['name']}"
        if route['start_port'].get('country'):
            start_popup += f" ({route['start_port']['country']})"
            
        folium.Marker(
            [route['start_port']['lat'], route['start_port']['lon']],
            popup=start_popup,
            icon=folium.Icon(color='green')
        ).add_to(m)
        
        # Add end port marker
        end_popup = f"End: {route['end_port']['name']}"
        if route['end_port'].get('country'):
            end_popup += f" ({route['end_port']['country']})"
            
        folium.Marker(
            [route['end_port']['lat'], route['end_port']['lon']],
            popup=end_popup,
            icon=folium.Icon(color='red')
        ).add_to(m)
        
        # Add route line if we have coordinates
        if route.get('route_coordinates'):
            # Convert route coordinates from (lon, lat) to (lat, lon) for folium
            route_points = [(coord[1], coord[0]) for coord in route['route_coordinates']]
            
            # Generate a random color for the route
            route_color = f"#{random.randint(0, 0xFFFFFF):06x}"
            
            folium.PolyLine(
                route_points,
                color=route_color,
                weight=2.5,
                opacity=0.8,
                popup=f"{route['vessel_id']}: {route['start_port']['name']} to {route['end_port']['name']}"
            ).add_to(m)
    
    # Save the map
    m.save(output_file)
    print(f"Map saved to {output_file}")
    return output_file

# Step 5: Save route data to CSV
def save_route_data(routes, output_file="vessel_routes.csv"):
    """Save route information to a CSV file."""
    routes_df = pd.DataFrame([
        {
            'vessel_id': route['vessel_id'],
            'start_port': route['start_port']['name'],
            'start_country': route['start_port'].get('country', ''),
            'start_lat': route['start_port']['lat'],
            'start_lon': route['start_port']['lon'],
            'end_port': route['end_port']['name'],
            'end_country': route['end_port'].get('country', ''),
            'end_lat': route['end_port']['lat'],
            'end_lon': route['end_port']['lon'],
            'route_points': len(route.get('route_coordinates', [])),
        }
        for route in routes
    ])
    
    routes_df.to_csv(output_file, index=False)
    print(f"Route data saved to {output_file}")
    return output_file

# Main function to execute the entire route generation process
def main(port_file, num_vessels=1):
    """Main function to execute the entire route generation process."""
    # Load port data
    print(f"Loading port data from {port_file}...")
    ports_df = load_port_data(port_file)
    
    if ports_df is None or len(ports_df) < 2:
        print("Error: Not enough valid ports in the data file.")
        return None, None
    
    # Generate random routes
    print(f"Generating routes for {num_vessels} vessels...")
    vessel_routes = generate_random_routes(ports_df, num_vessels)
    
    # Generate realistic sea routes
    print("Calculating realistic sea routes...")
    vessel_routes = generate_searoutes(vessel_routes)
    
    # Visualize the routes
    print("Creating visualization...")
    map_file = visualize_routes(vessel_routes)
    
    # Save route data
    data_file = save_route_data(vessel_routes)
    
    return vessel_routes, map_file, data_file

In [8]:
main(port_file="data/UpdatedPub150.csv", num_vessels=10)

Loading port data from data/UpdatedPub150.csv...
Loaded 3824 ports with valid coordinates
Generating routes for 10 vessels...
Generated route for Vessel_1: Sarnia to Greater Plutonio Terminal
Generated route for Vessel_2: Sokhumi to Vila Do Porto
Generated route for Vessel_3: Galeota Point Terminal to Mongla
Generated route for Vessel_4: Hareid to Mariners Harbor SI
Generated route for Vessel_5: Trieste to Tutoia
Generated route for Vessel_6: Raahe to Ballstad
Generated route for Vessel_7: Gingoog to Beaumont
Generated route for Vessel_8: Mata-Utu to Uglegorsk
Generated route for Vessel_9: Manokwari Road to Kanokawa Ko
Generated route for Vessel_10: Kwiguk to New Amsterdam
Calculating realistic sea routes...
Generating route from (-82.416667, 42.983333) to (12.1, -7.833333)...
Route generated for Vessel_1 from Sarnia to Greater Plutonio Terminal with 80 points
Generating route from (40.983333, 42.983333) to (-25.15, 36.933333)...
Route generated for Vessel_2 from Sokhumi to Vila Do Por

([{'vessel_id': 'Vessel_1',
   'start_port': {'name': 'Sarnia',
    'country': 'Canada',
    'lat': np.float64(42.983333),
    'lon': np.float64(-82.416667)},
   'end_port': {'name': 'Greater Plutonio Terminal',
    'country': 'Angola',
    'lat': np.float64(-7.833333),
    'lon': np.float64(12.1)},
   'route_coordinates': [[-82.428223, 43.002304],
    [-82.498535, 42.633105],
    [-83.006103, 42.360911],
    [-83.164307, 41.911045],
    [-82.722657, 41.648465],
    [-78.936768, 42.810497],
    [-79.125733, 43.420087],
    [-76.0901, 44.2466],
    [-75.4908, 44.7099],
    [-75.3311, 44.805],
    [-74.911652, 44.98617],
    [-74.509277, 45.066731],
    [-74.224544, 45.260854],
    [-73.939087, 45.299451],
    [-73.782806, 45.420073],
    [-73.572144, 45.427015],
    [-73.524902, 45.460319],
    [-73.535614, 45.509613],
    [-73.488373, 45.617586],
    [-73.373291, 45.793129],
    [-73.211243, 45.906207],
    [-73.164276, 46.038911],
    [-73.040098, 46.061274],
    [-72.806396, 46.21785

In [13]:
ports_df = pd.read_csv("data/UpdatedPub150.csv")

In [14]:
ports_df = ports_df[['Main Port Name', 'Latitude', 'Longitude', 'Country Code']]
ports_df.rename(columns={
            'Main Port Name': 'port_name',
            'Latitude': 'latitude',
            'Longitude': 'longitude',
            'Country Code': 'country'
        }, inplace=True)
        
# Filter out ports with missing coordinates
ports_dataframe = ports_df.dropna(subset=['latitude', 'longitude'])
ports_df.head()

,port_name,latitude,longitude,country
0,Maurer,40.533333,-74.250000,United States
1,Mangkasa Oil Terminal,-2.733333,121.066667,Indonesia
2,Iharana,-13.350000,50.000000,Madagascar
3,Andoany,-13.400000,48.300000,Madagascar
4,Chake Chake,-5.250000,39.766667,Tanzania


In [15]:
routes = []
num_vessels = 1
    

for vessel_id in range(1, num_vessels + 1):
    # Randomly select two different ports
    selected_ports = ports_dataframe.sample(n=2).reset_index(drop=True)
    
    start_port = {
        'name': selected_ports.loc[0, 'port_name'],
        'country': selected_ports.loc[0, 'country'] if 'country' in selected_ports.columns else '',
        'lat': selected_ports.loc[0, 'latitude'],
        'lon': selected_ports.loc[0, 'longitude']
    }
    
    end_port = {
        'name': selected_ports.loc[1, 'port_name'],
        'country': selected_ports.loc[1, 'country'] if 'country' in selected_ports.columns else '',
        'lat': selected_ports.loc[1, 'latitude'],
        'lon': selected_ports.loc[1, 'longitude']
    }
    
    routes.append({
        'vessel_id': f"Vessel_{vessel_id}",
        'start_port': start_port,
        'end_port': end_port
    })
    
    print(f"Generated route for Vessel_{vessel_id}: {start_port['name']} to {end_port['name']}")

Generated route for Vessel_1: Bristol to Ilwaco


In [16]:
for route in routes:
    start = (float(route['start_port']['lon']), float(route['start_port']['lat']))
    end = (float(route['end_port']['lon']), float(route['end_port']['lat']))
    route_geometry = searoute(start, end)
    route['route_coordinates'] = route_geometry['geometry']['coordinates']

In [17]:
map_center = [0, 0]
lats = []
lons = []
for route in routes:
    if route['route_coordinates']:
        for coord in route['route_coordinates']:
            lons.append(coord[0])
            lats.append(coord[1])
map_center = [sum(lats)/len(lats), sum(lons)/len(lons)]
map_center

[27.582405375, -93.94373098076923]

In [18]:
m = folium.Map(location=map_center, zoom_start=2)

for route in routes:
    # Add start port marker
    start_popup = f"Start: {route['start_port']['name']}"
    if route['start_port'].get('country'):
        start_popup += f" ({route['start_port']['country']})"
        
    folium.Marker(
        [route['start_port']['lat'], route['start_port']['lon']],
        popup=start_popup,
        icon=folium.Icon(color='green')
    ).add_to(m)
    
    # Add end port marker
    end_popup = f"End: {route['end_port']['name']}"
    if route['end_port'].get('country'):
        end_popup += f" ({route['end_port']['country']})"
        
    folium.Marker(
        [route['end_port']['lat'], route['end_port']['lon']],
        popup=end_popup,
        icon=folium.Icon(color='red')
    ).add_to(m)
    
    # Add route line if we have coordinates
    if route.get('route_coordinates'):
        # Convert route coordinates from (lon, lat) to (lat, lon) for folium
        route_points = [(coord[1], coord[0]) for coord in route['route_coordinates']]
        
        # Generate a random color for the route
        route_color = f"#{random.randint(0, 0xFFFFFF):06x}"
        
        folium.PolyLine(
            route_points,
            color=route_color,
            weight=2.5,
            opacity=0.8,
            popup=f"{route['vessel_id']}: {route['start_port']['name']} to {route['end_port']['name']}"
        ).add_to(m)

# Save the map
m.save("vessel_routes.html")